In [32]:
import os
import random
import sys
import numpy as np
from safetensors.torch import load_file
import torch
import math
from safetensors.torch import save_file

import time
start_time = time.time()
# ----------------------------------------------------------

method = 'Qwen2_5_0_5B_base'
safetensors_path_act = f'saved_tensor/{method}/activation.safetensors'
state_dict_qwt_input = load_file(safetensors_path_act)

In [33]:
for k, v in state_dict_qwt_input.items():
    print(k, v.shape)

layers.0.input torch.Size([1, 2048, 896])
layers.1.input torch.Size([1, 2048, 896])
layers.10.input torch.Size([1, 2048, 896])
layers.11.input torch.Size([1, 2048, 896])
layers.12.input torch.Size([1, 2048, 896])
layers.13.input torch.Size([1, 2048, 896])
layers.14.input torch.Size([1, 2048, 896])
layers.15.input torch.Size([1, 2048, 896])
layers.16.input torch.Size([1, 2048, 896])
layers.17.input torch.Size([1, 2048, 896])
layers.18.input torch.Size([1, 2048, 896])
layers.19.input torch.Size([1, 2048, 896])
layers.2.input torch.Size([1, 2048, 896])
layers.20.input torch.Size([1, 2048, 896])
layers.21.input torch.Size([1, 2048, 896])
layers.22.input torch.Size([1, 2048, 896])
layers.23.input torch.Size([1, 2048, 896])
layers.3.input torch.Size([1, 2048, 896])
layers.4.input torch.Size([1, 2048, 896])
layers.5.input torch.Size([1, 2048, 896])
layers.6.input torch.Size([1, 2048, 896])
layers.7.input torch.Size([1, 2048, 896])
layers.8.input torch.Size([1, 2048, 896])
layers.9.input torch

In [ ]:
import torch
import torch.nn as nn

# 实例化MSELoss对象
loss = nn.MSELoss()

In [ ]:
for a, b in zip(state_dict_base.values(), state_dict_qwt_input.values()):
    print(a.equal(b)) #, a.view(-1)[:5], b.view(-1)[:5])

In [ ]:
with open('log/123.log', 'w') as f:
    for idx, (a, b, c, d, e) in enumerate(zip(state_dict_base.values(), state_dict_quant.values(), state_dict_base.values(), state_dict_clamp.values(), state_dict_clamp_qwt.values())):
        f.writelines(f'{idx} base-quant: {loss(a.float(), b.float()).item():>8.4f}, base-clamp: {loss(c.float(), d.float()).item():>8.4f}, base-qwt: {loss(a.float(), e.float()).item():>8.4f}\n') # 11, 16

In [ ]:
for idx in range(22): 
    print(idx, state_dict_base[f'base_layers.{idx}.input'].equal(state_dict_qwt[f'qwt_layers.{idx}.input_full']))
    # print(idx, loss(state_dict_base[f'base_layers.{idx}.input'].float(), state_dict_qwt[f'qwt_layers.{idx}.input_full'].float()))
    # print('='*10)
    # print(state_dict_base[f'base_layers.{idx}.input'].view(-1)[:5])
    # print(state_dict_qwt[f'qwt_layers.{idx}.input_full'].view(-1)[:5])


In [ ]:
def tensor_gpu_memory(tensor, name=None):
    if tensor.is_cuda:
        size_in_bytes = tensor.element_size() * tensor.numel()
        size_in_MB = size_in_bytes / 1024**2
        print(f"{name if name else ''} size: {size_in_MB:.2f} MB")
    else:
        print(f"{name if name else ''} is not on GPU.")

In [ ]:
import torch
tensor = torch.empty((512, 2048, 4096), dtype=torch.float32).to('cuda')
# print((4 * 512 * 2048 * 4096 * 32) / 1024 / 1024 / 1024 / 1024)

In [ ]:
'''
to speedup debug, set length=512, n_sample=1, use opt-125m
开源代码量化后的结果向中间结果做线性回归，此版本修改为向全精度对齐
'''
import os
import shutil
import sys
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from tqdm import tqdm
import torch.distributed as dist


import argparse
import torch.nn.functional as F
from torch.utils.data import Dataset
import time
from datetime import datetime
import sys
from globalVar import (get_save_tensor_enable,
                       save_tensors,
                       set_save_tensor_enable,
                       set_data_type,
                       append_activation,
                       set_profiling_enable)

def gather_tensor_from_multi_processes(input, world_size):
    if world_size == 1:
        return input
    torch.cuda.synchronize()
    dist.all_gather(gathered_tensors, input)
    gathered_tensors = torch.cat(gathered_tensors, dim=0)
    torch.cuda.synchronize()

    return gathered_tensors
def lienar_regression(X, Y, block_id=0):
    X = X.reshape(-1, X.size(-1)).float()

    X = gather_tensor_from_multi_processes(X, world_size=args.world_size)

    X_add_one = torch.cat([X, torch.ones(size=[X.size(0), ], device=X.device).reshape(-1, 1)], dim=-1)
    Y = Y.reshape(-1, Y.size(-1)).float()

    Y = gather_tensor_from_multi_processes(Y, world_size=args.world_size)

    # _write('the shape of X_add_one is {}, Y is {}'.format(X_add_one.size(), Y.size()))

    X_add_one_T = X_add_one.t()
    W_overall = torch.inverse(X_add_one_T @ X_add_one) @ X_add_one_T @ Y

    W = W_overall[:-1, :]
    b = W_overall[-1, :]

    Y_pred = X @ W + b

    abs_loss = (Y - Y_pred).abs().mean()

    ss_tot = torch.sum((Y - Y.mean(dim=0)).pow(2))
    ss_res = torch.sum((Y - Y_pred).pow(2))
    r2_score = 1 - ss_res / ss_tot
    # _write('block : {}      abs : {:.6f}      r2 : {:.3f}'.format(block_id, abs_loss, r2_score))
    return W, b, r2_score
def tensor_gpu_memory(tensor, name=None):
    if tensor.is_cuda:
        size_in_bytes = tensor.element_size() * tensor.numel()
        size_in_MB = size_in_bytes / 1024**2
        print(f"{name if name else ''} size: {size_in_MB:.2f} MB")
    else:
        print(f"{name if name else ''} is not on GPU.")
# current date and time
current_time = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
start_time = time.time()
# ----------------------------------------------------------
parser = argparse.ArgumentParser()
parser.add_argument("--alpha", type=float, default=0.5)
parser.add_argument("--model_name", type=str, default="opt-125m")
parser.add_argument(
    "--act_scales_path",
    type=str,
    default="act_scales/llama-2-7b.pt",
)
parser.add_argument("--n_samples", type=int, default=None)
parser.add_argument("--device", type=str, default='cuda')
parser.add_argument('--start_block', default=0, type=int)
parser.add_argument("--local-rank", default=0, type=int)
parser.add_argument("--batch_size", default=32, type=int)
parser.add_argument("--num_workers", default=4, type=int)
parser.add_argument("--eval_base", action="store_true")
parser.add_argument("--eval_quant", action="store_true")
parser.add_argument("--eval_clamp", action="store_true")
parser.add_argument("--eval_quant_qwt", action="store_true")
parser.add_argument("--eval_clamp_qwt", action="store_true")
parser.add_argument("--profiling", action="store_true")
args = parser.parse_args()

class Evaluator:
    def __init__(self, dataset, tokenizer, device, n_samples=40):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        self.dataset = tokenizer(
            "\n\n".join(dataset["text"]), return_tensors="pt"
        ).input_ids.to(device)

        self.n_samples = n_samples

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        nlls = []
        length = 2048
        n_samples = self.n_samples if self.n_samples else self.dataset.size(1) // length
        for i in tqdm(range(n_samples), desc="Evaluating..."):
            batch = self.dataset[:, (i * length) : ((i + 1) * length)].to(model.device)
            with torch.no_grad():
                lm_logits = model(batch).logits
            shift_logits = lm_logits[:, :-1, :].contiguous().float()
            shift_labels = self.dataset[:, (i * length) : ((i + 1) * length)][:, 1:]
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1)
            )
            neg_log_likelihood = loss.float() * length
            nlls.append(neg_log_likelihood)

        return torch.exp(torch.stack(nlls).sum() / (n_samples * length))

args.world_size = 1
args.rank = 0  # global rank
alpha = args.alpha
model_path = '/localssd/lbxj/' + args.model_name
# model_path = '/cephfs/juxin/models/' + args.model_name
act_scales_path = args.act_scales_path
n_samples = args.n_samples
train_samples = 64 # 64
# set_save_tensor_enable()

